# Identifying (volatility-)regimes in the the EUR/USD spot exchange rate using clustering algorithms: An Oil and Gas Perspective on Parity Conditions.
Seminar in Applied Financial Economics: Applied Econometrics of FX Markets - Prof. Dr. Reitz
<br>
**Josef Fella and Robert Hennings**
<br>
Christian Albrechts University of Kiel
<br>
*josef.fella@stu.uni-kiel.de and robert.hennings@stu.uni-kiel.de*
<br>
GitHub: https://github.com/RobertHennings/Seminar
<br>
Kiel - 14.11.2025

## Outline
1. Research Hypothesis
<br>
1.1 Energy Commodity Price Shocks: The Pass-Through Effect and Implications for Monetary Policy
<br>
1.2 Formulated Research Hypothesis
<br>
2. Theoretical Framework
<br>
2.1 Impact of Inflation on Measurements: What are prices and how are they measured?
<br>
2.2 A simple model of exchange rates and commodity prices
<br>
2.3. Theoretical Framework
<br>
3. Model Results
<br>
3.1 Regime identification - Model comparison and selection
<br>
4. Conclusion and Discussion
<br>
4.1 Seminar Project Summary
<br>
4.2 Seminar Project Limitations
<br>
4.3 Future Research
<br>
5. Appendix
<br>
5.1 Abbreviations
<br>
5.2 Systematic Literature Overview: Main Approaches
<br>
5.3 Figures and Tables
<br>
5.3 Data and Definitions
<br>

## Short description of the notebook contents
The contents of this Jupyter notebook produce the main results for the chapter Model Results. It is based on the file model_benchmark.py, that includes all the detailed data generating procedures, what have been skipped here in order to not cause confusion, if data can't be appropriatley loaded from the various sources due to a number of different potential reasons.
<br>
In this notebook the defined bechmark models are fitted to the data and the main inference is extracted. These are the standard UIP-regression and the Markov-Switching models using different datasets.

Import dependencies/packages:

In [1]:
import os
import pandas as pd
import numpy as np

Set global config settings:

**!!!!CHANGE WORKING DIRECTORY HERE!!!!**

In [ ]:
SEMINAR_PATH = r"/Users/Robert_Hennings/Uni/Master/Seminar"
# Example: r"/Users/Robert_Hennings/Uni/Master/Seminar"

In [ ]:
SEMINAR_CODE_PATH = rf"{SEMINAR_PATH}/src/seminar_code"
MODELS_PATH = rf"{SEMINAR_CODE_PATH}/models"
FIGURES_PATH = rf"{SEMINAR_PATH}/reports/figures"
TABLES_PATH = rf"{SEMINAR_PATH}/reports/tables"
DATA_PATH = rf"{SEMINAR_PATH}/data"
PRESENTATION_DATA = rf"{SEMINAR_PATH}/reports/presentation_latex_version/data"

# Change working directory to seminar code path
print(os.getcwd())
os.chdir(SEMINAR_CODE_PATH) # <- needed to be able to import the ModelObject class from utils
print(os.getcwd())

/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/model
/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code


## Benchmark Models for Regime Identification - Basic Full time UIP Regression - BIS Central Bank Policy Rates

In [22]:
# Define a simple wrapper function for the statsmodels OLS regression
def run_uip_regression(
        dep_var: str,
        indep_var: str,
        data: pd.DataFrame,
        cov_type: str="nonrobust",
        use_t: bool=True
        ):
    import statsmodels.api as sm
    X = sm.add_constant(data[indep_var])
    y = data[dep_var]
    model = sm.OLS(y, X).fit(cov_type=cov_type, use_t=use_t)
    return model

In [23]:
file_name = r"chap_04_uip_data_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
uip_data_df = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded UIP data:\n{uip_data_df.head(n=10)}")

Loaded UIP data:
                  EUR   WTI Oil   Nat Gas  i_diff_EUR
TIME_PERIOD                                          
1999-01-05   0.004412  0.031074  0.024098        1.75
1999-01-06   0.010600 -0.064331  0.004890        1.75
1999-01-07  -0.003089 -0.011615  0.065847        1.75
1999-01-08   0.010161 -0.005374  0.005249        1.75
1999-01-11   0.001733 -0.027937  0.037538        1.75
1999-01-12  -0.001213  0.039489  0.005479        1.75
1999-01-13  -0.012906  0.045967 -0.027102        1.75
1999-01-14   0.000770  0.008143  0.054959        1.75
1999-01-15   0.008419  0.001637 -0.005634        1.75
1999-01-19  -0.001638  0.006574  0.005634        1.75


**!!!NOTE: Use robust standard-errors in the UIP-regression!!!**

In [24]:
# First run the UIP regression on the full sample
full_sample_uip_results = {}
currency_pairs = ['EUR']
for currency in currency_pairs:
    dep_var = f'{currency}'
    indep_var = f'i_diff_{currency}'
    model = run_uip_regression(dep_var, indep_var, uip_data_df, cov_type="HC1")
    full_sample_uip_results[currency] = model
# Print the summary of the full sample UIP regressions
for currency, model in full_sample_uip_results.items():
    print(f"Full Sample UIP Regression Results for {currency}/USD:")
    print(model.summary())
    print("\n")

Full Sample UIP Regression Results for EUR/USD:
                            OLS Regression Results                            
Dep. Variable:                    EUR   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1223
Date:                Sat, 08 Nov 2025   Prob (F-statistic):              0.727
Time:                        12:16:07   Log-Likelihood:                 24721.
No. Observations:                6637   AIC:                        -4.944e+04
Df Residuals:                    6635   BIC:                        -4.942e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons

In [8]:
# The correct Hypothesis for the tested parameters are:
# H0: β0 = 0 (no constant term), ß1 = 1 (interest rate differential fully explains exchange rate changes)
# H1: β0 ≠ 0, ß1 ≠ 1
estimated_params_df = pd.DataFrame(model.summary().tables[1].data)
estimated_params_df.columns = estimated_params_df.iloc[0]
estimated_params_df = estimated_params_df[1:]
# Transfer all columns to numeric where possible
estimated_params_df = estimated_params_df.apply(pd.to_numeric, errors='ignore')
# Therefore we have to adjust the t-test and p-values accordingly
from scipy import stats
corrected_t_i_diff = (estimated_params_df["coef"][2] - 1.0) / estimated_params_df["std err"][2]
corrected_p_i_diff = 2 * (1 - stats.t.cdf(np.abs(corrected_t_i_diff), df=model.df_resid))
print(f"Corrected t-statistic for i_diff: {corrected_t_i_diff}")
print(f"Corrected p-value for i_diff: {corrected_p_i_diff}")

Corrected t-statistic for i_diff: -16128.682741935483
Corrected p-value for i_diff: 0.0


/var/folders/5j/5rywkp2x7zs75k_ml53cn10w0000gn/T/ipykernel_67003/3688584732.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  estimated_params_df = estimated_params_df.apply(pd.to_numeric, errors='ignore')


## Benchmark Models for Regime Identification - Basic Full time UIP Regression - 3M Interbank Lending Rates

In [10]:
file_name = r"chap_04_uip_data_df_3m_interbank_lending_rates.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
uip_data_df = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded UIP data:\n{uip_data_df.head(n=10)}")

Loaded UIP data:
                 EUR  i_diff_EUR
1999-01-05 -0.004412    0.461992
1999-01-06 -0.010600    0.463243
1999-01-07  0.003089    0.463750
1999-01-08 -0.010161    0.460078
1999-01-11 -0.001733    0.463243
1999-01-12  0.001213    0.468280
1999-01-13  0.012906    0.462930
1999-01-14 -0.000770    0.459688
1999-01-15 -0.008419    0.460740
1999-01-19  0.001638    0.480000


**!!!NOTE: Use robust standard-errors in the UIP-regression!!!**

In [11]:
full_sample_uip_results = {}
currency_pairs = ['EUR']
for currency in currency_pairs:
    dep_var = f'{currency}'
    indep_var = f'i_diff_{currency}'
    model = run_uip_regression(dep_var, indep_var, uip_data_df, cov_type="HC1")
    full_sample_uip_results[currency] = model
# Print the summary of the full sample UIP regressions
for currency, model in full_sample_uip_results.items():
    print(f"Full Sample UIP Regression Results for {currency}/USD:")
    print(model.summary())
    print("\n")

Full Sample UIP Regression Results for EUR/USD:
                            OLS Regression Results                            
Dep. Variable:                    EUR   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9144
Date:                Sat, 08 Nov 2025   Prob (F-statistic):              0.339
Time:                        11:41:47   Log-Likelihood:                 18423.
No. Observations:                5021   AIC:                        -3.684e+04
Df Residuals:                    5019   BIC:                        -3.683e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons

In [12]:
# The correct Hypothesis for the tested parameters are:
# H0: β0 = 0 (no constant term), ß1 = 1 (interest rate differential fully explains exchange rate changes)
# H1: β0 ≠ 0, ß1 ≠ 1
estimated_params_df = pd.DataFrame(model.summary().tables[1].data)
estimated_params_df.columns = estimated_params_df.iloc[0]
estimated_params_df = estimated_params_df[1:]
# Transfer all columns to numeric where possible
estimated_params_df = estimated_params_df.apply(pd.to_numeric, errors='ignore')
# Therefore we have to adjust the t-test and p-values accordingly
from scipy import stats
corrected_t_i_diff = (estimated_params_df["coef"][2] - 1.0) / estimated_params_df["std err"][2]
corrected_p_i_diff = 2 * (1 - stats.t.cdf(np.abs(corrected_t_i_diff), df=model.df_resid))
print(f"Corrected t-statistic for i_diff: {corrected_t_i_diff}")
print(f"Corrected p-value for i_diff: {corrected_p_i_diff}")

Corrected t-statistic for i_diff: -inf
Corrected p-value for i_diff: 0.0


/var/folders/5j/5rywkp2x7zs75k_ml53cn10w0000gn/T/ipykernel_67003/3688584732.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  estimated_params_df = estimated_params_df.apply(pd.to_numeric, errors='ignore')
/var/folders/5j/5rywkp2x7zs75k_ml53cn10w0000gn/T/ipykernel_67003/3688584732.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  corrected_t_i_diff = (estimated_params_df["coef"][2] - 1.0) / estimated_params_df["std err"][2]


## Benchmark Models for Regime Identification - Markov Switching Model - Standard - Central Bank Policy Rates - Model B1

In [13]:
file_name = r"chap_04_uip_data_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
uip_data_df = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded UIP data:\n{uip_data_df.head(n=10)}")

Loaded UIP data:
                  EUR   WTI Oil   Nat Gas  i_diff_EUR
TIME_PERIOD                                          
1999-01-05   0.004412  0.031074  0.024098        1.75
1999-01-06   0.010600 -0.064331  0.004890        1.75
1999-01-07  -0.003089 -0.011615  0.065847        1.75
1999-01-08   0.010161 -0.005374  0.005249        1.75
1999-01-11   0.001733 -0.027937  0.037538        1.75
1999-01-12  -0.001213  0.039489  0.005479        1.75
1999-01-13  -0.012906  0.045967 -0.027102        1.75
1999-01-14   0.000770  0.008143  0.054959        1.75
1999-01-15   0.008419  0.001637 -0.005634        1.75
1999-01-19  -0.001638  0.006574  0.005634        1.75


In [14]:
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
msm = MarkovRegression(
    endog=uip_data_df['EUR'],
    exog=uip_data_df[['i_diff_EUR']],
    k_regimes=2,
    trend='c',  # or 'nc' for no constant
    switching_trend=True,
    switching_exog=True,
    switching_variance=True,
)
msm_fit = msm.fit(em_iter=10, search_reps=20)
print("Markov-Switching UIP Regression Results for EUR/USD:")
print(msm_fit.summary())

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Markov-Switching UIP Regression Results for EUR/USD:
                        Markov Switching Model Results                        
Dep. Variable:                    EUR   No. Observations:                 6637
Model:               MarkovRegression   Log Likelihood               25136.177
Date:                Sat, 08 Nov 2025   AIC                         -50256.353
Time:                        11:43:37   BIC                         -50201.950
Sample:                             0   HQIC                        -50237.556
                               - 6637                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001      0.000      0.839      0.402      -0.000       0.000

## Benchmark Models for Regime Identification - Markov Switching Model - Oil, Gas added - Central Bank Policy Rates - Model B1

In [15]:
file_name = r"chap_04_uip_data_df_central_bank_policy_rates_oil_gas_rol_vol_b1.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
uip_data_df = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded UIP data:\n{uip_data_df.head(n=10)}")

Loaded UIP data:
             EUR/USD  i_diff_EUR   WTI Oil   Nat Gas  WTI Oil TV  Nat Gas TV
1999-02-17 -0.003658        1.75  0.026891  0.024763       58609       22752
1999-02-18  0.000624        1.75  0.027469  0.024579       55270       46451
1999-02-19  0.013275        1.75  0.024727  0.024580       41229       21272
1999-02-22  0.003437        1.75  0.024806  0.021726       35580       48415
1999-02-23  0.003995        1.75  0.025894  0.021781       82882       46144
1999-02-24  0.001912        1.75  0.025314  0.020845       65511       68285
1999-02-25 -0.008711        1.75  0.024405  0.022870       67581       33960
1999-02-26  0.006617        1.75  0.023310  0.022131       64878       20543
1999-03-01  0.009504        1.75  0.023265  0.020169       43553       37075
1999-03-02 -0.003666        1.75  0.023910  0.020290       48377       28113


In [16]:
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
msm = MarkovRegression(
    endog=uip_data_df['EUR/USD'],
    exog=uip_data_df[['i_diff_EUR', 'WTI Oil', 'Nat Gas', "WTI Oil TV", "Nat Gas TV"]],
    k_regimes=2,
    trend='c',  # or 'nc' for no constant
    switching_trend=True,
    switching_exog=True,
    switching_variance=True,
)
msm_fit = msm.fit(em_iter=100, search_reps=200)
print("Markov-Switching UIP Regression Results for EUR/USD:")
print(msm_fit.summary())

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Library/Frameworks/Python.framework/Version

Markov-Switching UIP Regression Results for EUR/USD:
                        Markov Switching Model Results                        
Dep. Variable:                EUR/USD   No. Observations:                 6602
Model:               MarkovRegression   Log Likelihood                     nan
Date:                Sat, 08 Nov 2025   AIC                                nan
Time:                        11:44:13   BIC                                nan
Sample:                             0   HQIC                               nan
                               - 6602                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0038        nan        nan        nan         nan         nan

NOTE:
<br>
<br>
In our analysis, we encountered here, that the maximization of the likelihood failed when we include multiple external explanatory variables, we don't know the exact reason, but potential help could be offered by feature engineering the input data (i.e. standardisation techniques).

## Benchmark Models for Regime Identification - Markov Switching Model - Standard - 3M Interbank rates - Model B2

In [17]:
file_name = r"chap_04_uip_data_df_3m_interbank_lending_rates_b2.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
uip_data_df = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded UIP data:\n{uip_data_df.head(n=10)}")

Loaded UIP data:
             EUR/USD  i_diff_EUR
1999-01-05 -0.004412    0.461992
1999-01-06 -0.010600    0.463243
1999-01-07  0.003089    0.463750
1999-01-08 -0.010161    0.460078
1999-01-11 -0.001733    0.463243
1999-01-12  0.001213    0.468280
1999-01-13  0.012906    0.462930
1999-01-14 -0.000770    0.459688
1999-01-15 -0.008419    0.460740
1999-01-19  0.001638    0.480000


In [18]:
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
msm = MarkovRegression(
    endog=uip_data_df['EUR/USD'],
    exog=uip_data_df[['i_diff_EUR']],
    k_regimes=2,
    trend='c',  # or 'nc' for no constant
    switching_trend=True,
    switching_exog=True,
    switching_variance=True,
)
msm_fit = msm.fit(em_iter=10, search_reps=20)
print("Markov-Switching UIP Regression Results for EUR/USD:")
print(msm_fit.summary())

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Markov-Switching UIP Regression Results for EUR/USD:
                        Markov Switching Model Results                        
Dep. Variable:                EUR/USD   No. Observations:                 5021
Model:               MarkovRegression   Log Likelihood               18662.108
Date:                Sat, 08 Nov 2025   AIC                         -37308.216
Time:                        11:44:34   BIC                         -37256.045
Sample:                             0   HQIC                        -37289.935
                               - 5021                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.000      3.086      0.002       0.000       0.001

## Benchmark Models for Regime Identification - Markov Switching Model - Oil, Gas added - 3M Interbank rates - Model B2

In [19]:
file_name = r"chap_04_uip_data_df_3m_interbank_lending_rates_oil_gas_rol_vol_model_b2.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
uip_data_df = pd.read_excel(full_file_path, index_col=0)
print(f"Loaded UIP data:\n{uip_data_df.head(n=10)}")

Loaded UIP data:
             EUR/USD  i_diff_EUR  WTI Oil TV  Nat Gas TV   WTI Oil   Nat Gas
1999-02-17  0.003658    0.478750       58609       22752  0.026891  0.024763
1999-02-18 -0.000624    0.480155       55270       46451  0.027469  0.024579
1999-02-19 -0.013275    0.480467       41229       21272  0.024727  0.024580
1999-02-22 -0.003437    0.480780       35580       48415  0.024806  0.021726
1999-02-23 -0.003995    0.480625       82882       46144  0.025894  0.021781
1999-02-24 -0.001912    0.480938       65511       68285  0.025314  0.020845
1999-02-25  0.008711    0.482655       67581       33960  0.024405  0.022870
1999-02-26 -0.006617    0.481875       64878       20543  0.023310  0.022131
1999-03-01 -0.009504    0.481875       43553       37075  0.023265  0.020169
1999-03-02  0.003666    0.479727       48377       28113  0.023910  0.020290


In [20]:
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
msm = MarkovRegression(
    endog=uip_data_df['EUR/USD'],
    exog=uip_data_df[['i_diff_EUR', "WTI Oil", "Nat Gas", 'WTI Oil TV', 'Nat Gas TV']],
    k_regimes=2,
    trend='c',  # or 'nc' for no constant
    switching_trend=True,
    switching_exog=True,
    switching_variance=True,
)
msm_fit = msm.fit(em_iter=10, search_reps=20)
print("Markov-Switching UIP Regression Results for EUR/USD:")
print(msm_fit.summary())

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Library/Frameworks/Python.framework/Version

Markov-Switching UIP Regression Results for EUR/USD:
                        Markov Switching Model Results                        
Dep. Variable:                EUR/USD   No. Observations:                 4912
Model:               MarkovRegression   Log Likelihood                     nan
Date:                Sat, 08 Nov 2025   AIC                                nan
Time:                        11:45:02   BIC                                nan
Sample:                             0   HQIC                               nan
                               - 4912                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0035        nan        nan        nan         nan         nan

NOTE:
<br>
<br>
In our analysis, we encountered here, that the maximization of the likelihood failed when we include multiple external explanatory variables, we don't know the exact reason, but potential help could be offered by feature engineering the input data (i.e. standardisation techniques).